In [14]:
import pandas as pd
# import glob

# # Get a list of all CSV files in the directory
# csv_files = glob.glob('data/*.csv')

# # Create an empty DataFrame to store the combined data
# combined_data = pd.DataFrame()

# # Iterate over the CSV files and concatenate their data horizontally
# for file in csv_files:
#     data = pd.read_csv(file)
#     combined_data = pd.concat([combined_data, data], axis=1)

# # Save the combined data to a new CSV file
# combined_data.to_csv('combined_data.csv', index=False)

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import sklearn.metrics
from sklearn.metrics import accuracy_score
# Load the dataset
data = pd.read_csv('combined_data.csv')
data.head()

updrs = data['updrs_1'] + data['updrs_2'] + data['updrs_3'] + data['updrs_4'];
data['updrs'] = updrs
data.fillna(0, inplace=True)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('On', 1)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('Off', 0)
print(data.head())
# Select the relevant features
selected_features = ['PeptideAbundance', 'NPX', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4','upd23b_clinical_state_on_medication']
X = data[selected_features]
y = data['updrs']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the regression model
model = RandomForestRegressor().fit(X_train, y_train)


# Make predictions on the test set
y_pred = model.predict(X_test)
# Calculate MSE on the test set
mse = mean_squared_error(y_test, y_pred)
# Evaluate the model

# Add your evaluation metrics here, such as mean squared error or R-squared
print("Mean Squared Error (MSE):", mse)
y_pred_rounded = [round(value) for value in y_pred]

# Compute accuracy score
accuracy = accuracy_score(y_test, y_pred_rounded)

print("Accuracy:", accuracy)

/var/folders/n8/8wwrm4fd5138rt4fqtxgcp3r0000gn/T/ipykernel_53495/565677968.py:9: DtypeWarning: Columns (0,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('combined_data.csv')


  visit_id  patient_id  visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
0     55_0        55.0          0.0     10.0      6.0     15.0      0.0   
1     55_3        55.0          3.0     10.0      7.0     25.0      0.0   
2     55_6        55.0          6.0      8.0     10.0     34.0      0.0   
3     55_9        55.0          9.0      8.0      9.0     30.0      0.0   
4    55_12        55.0         12.0     10.0     10.0     41.0      0.0   

   upd23b_clinical_state_on_medication visit_id.1  visit_month.1  \
0                                    0       55_0            0.0   
1                                    0       55_0            0.0   
2                                    0       55_0            0.0   
3                                    1       55_0            0.0   
4                                    1       55_0            0.0   

   patient_id.1 UniProt       NPX visit_id.2  visit_month.2  patient_id.2  \
0          55.0  O00391   11254.3       55_0              0    

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from dbn.tensorflow import SupervisedDBNRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import sklearn.metrics

# Load the dataset
data = pd.read_csv('combined_data.csv')
data.head()

updrs = data['updrs_1'] + data['updrs_2'] + data['updrs_3'] + data['updrs_4']
data['updrs'] = updrs
data.fillna(0, inplace=True)

data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('On', 1)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('Off', 0)

# Select the relevant features
selected_features = ['PeptideAbundance', 'NPX', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4','upd23b_clinical_state_on_medication']
X = data[selected_features]
y = data['updrs']

# Preprocess data
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

# Training
regressor = SupervisedDBNRegression(hidden_layers_structure=[100],
                                    learning_rate_rbm=0.01,
                                    learning_rate=0.01,
                                    n_epochs_rbm=10,
                                    n_iter_backprop=100,
                                    batch_size=32,
                                    activation_function='relu',
                                    dropout_p=0.2)
regressor.fit(X_train, y_train)

# Test
y_pred = regressor.predict(X_test)

# Calculate MSE on the test set
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Accuracy isn't typically the best metric for regression problems.
# However, if you still want to compute it, you can convert the outputs to class labels and compute the accuracy
y_pred_rounded = [round(value[0]) for value in y_pred]
y_test_rounded = [round(value) for value in y_test]
accuracy = accuracy_score(y_test_rounded, y_pred_rounded)

print("Accuracy:", accuracy)


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# Load the dataset
data = pd.read_csv('combined_data.csv')

updrs = data['updrs_1'] + data['updrs_2'] + data['updrs_3'] + data['updrs_4']
data['updrs'] = updrs
data.fillna(0, inplace=True)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('On', 1)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('Off', 0)

# Select the relevant features
selected_features = ['PeptideAbundance', 'NPX', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4', 'upd23b_clinical_state_on_medication']

X = data[selected_features].values
y = data['updrs'].values

# Scaling the inputs
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create feature columns for input
feat_cols = [tf.feature_column.numeric_column("x", shape=[X_train.shape[1]])]

# DNNRegressor model setup
regressor = tf.estimator.DNNRegressor(
    feature_columns=feat_cols, 
    hidden_units=[10,10],
    model_dir="/tmp/mymodel"
)

# input function for training
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=None,
    shuffle=True
)

# training the model
regressor.train(input_fn=train_input_fn, steps=2000)

# input function for testing
test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
)

# predicting the values
y_pred_generator = regressor.predict(input_fn=test_input_fn)
y_pred = [y['predictions'][0] for y in y_pred_generator]

# Mean Squared Error calculation
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)


/var/folders/n8/8wwrm4fd5138rt4fqtxgcp3r0000gn/T/ipykernel_92831/1146985350.py:7: DtypeWarning: Columns (0,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('combined_data.csv')


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mymodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_expe

AttributeError: module 'inspect' has no attribute 'getargspec'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
import numpy as np
from keras.optimizers import SGD
# Load the dataset
data = pd.read_csv('combined_data.csv')
data.head()

updrs = data['updrs_1'] + data['updrs_2'] + data['updrs_3'] + data['updrs_4']
data['updrs'] = updrs
data.fillna(0, inplace=True)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('On', 1)
data['upd23b_clinical_state_on_medication'] = data['upd23b_clinical_state_on_medication'].replace('Off', 0)
print(data.head())

# Select the relevant features
selected_features = ['PeptideAbundance', 'NPX', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4','upd23b_clinical_state_on_medication']
X = data[selected_features]
y = data['updrs']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the RBM and DBN classes as shown in the previous code snippet
class RBM:
    def __init__(self, input_shape, output_shape, learning_rate=0.1, batch_size=10, epochs=50):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.rbm_model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(self.output_shape, activation='sigmoid', input_shape=(self.input_shape,)))
        model.add(Dense(self.input_shape, activation='sigmoid'))
        return model

    def train(self, train_data):
        self.rbm_model.compile(optimizer=SGD(self.learning_rate), loss='mean_squared_error')
        x_train = np.array(train_data)
        self.rbm_model.fit(x_train, x_train, batch_size=self.batch_size, epochs=self.epochs, verbose=True)

    def get_weights(self):
        return self.rbm_model.layers[0].get_weights()

class DBN:
    def __init__(self, rbm_list):
        self.rbm_list = rbm_list

    def pretrain(self, train_data):
        temp_train = train_data
        for rbm in self.rbm_list:
            print("Training the next RBM layer")
            rbm.train(temp_train)
            weights, biases = rbm.get_weights()
            temp_train = np.tanh(np.dot(temp_train, weights) + biases) 
# Initialize the RBMs and the DBN
rbm1 = RBM(input_shape=X_train.shape[1], output_shape=512)
rbm2 = RBM(input_shape=512, output_shape=128)

dbn = DBN([rbm1, rbm2])

# Pretrain the DBN
dbn.pretrain(X_train)

# Build the regression model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate MSE on the test set
mse = mean_squared_error(y_test, y_pred)

# Print the MSE
print("Mean Squared Error (MSE):", mse)


/var/folders/n8/8wwrm4fd5138rt4fqtxgcp3r0000gn/T/ipykernel_92831/3720819062.py:10: DtypeWarning: Columns (0,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('combined_data.csv')


  visit_id  patient_id  visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
0     55_0        55.0          0.0     10.0      6.0     15.0      0.0   
1     55_3        55.0          3.0     10.0      7.0     25.0      0.0   
2     55_6        55.0          6.0      8.0     10.0     34.0      0.0   
3     55_9        55.0          9.0      8.0      9.0     30.0      0.0   
4    55_12        55.0         12.0     10.0     10.0     41.0      0.0   

   upd23b_clinical_state_on_medication visit_id.1  visit_month.1  \
0                                    0       55_0            0.0   
1                                    0       55_0            0.0   
2                                    0       55_0            0.0   
3                                    1       55_0            0.0   
4                                    1       55_0            0.0   

   patient_id.1 UniProt       NPX visit_id.2  visit_month.2  patient_id.2  \
0          55.0  O00391   11254.3       55_0              0    

Training the next RBM layer
Epoch 1/50
78547/78547 [==============================] - 47s 592us/step - loss: 0.5448
Epoch 2/50
78547/78547 [==============================] - 46s 588us/step - loss: 0.5409
Epoch 3/50
78547/78547 [==============================] - 47s 593us/step - loss: 0.5408
Epoch 4/50
78547/78547 [==============================] - 47s 597us/step - loss: 0.5408
Epoch 5/50
78547/78547 [==============================] - 46s 587us/step - loss: 0.5408
Epoch 6/50
78547/78547 [==============================] - 46s 590us/step - loss: 0.5408
Epoch 7/50
78547/78547 [==============================] - 46s 591us/step - loss: 0.5408
Epoch 8/50
78547/78547 [==============================] - 47s 603us/step - loss: 0.5408
Epoch 9/50
78547/78547 [==============================] - 49s 621us/step - loss: 0.5408
Epoch 10/50
78547/78547 [==============================] - 48s 610us/step - loss: 0.5408
Epoch 11/50
78547/78547 [==============================] - 46s 591us/step - loss: 0.5408
Ep